# Module loading

In [ ]:
library(IRdisplay)
library(readr)
library(fgsea)
library(dplyr)
library(reticulate)

In [ ]:
pd <- import("pandas")

# Settings

In [ ]:
OUTPUT_DIR <- Sys.getenv("PHENOPLIER_RESULTS_CRISPR_ANALYSES_BASE_DIR")

In [ ]:
OUTPUT_DIR

In [ ]:
dir.create(OUTPUT_DIR, recursive=TRUE)

# Data loading

## Lipids gene sets

In [ ]:
input_file <- Sys.getenv("PHENOPLIER_CRISPR_LIPIDS_GENE_SETS_FILE")
display(input_file)

In [ ]:
all_genes_ranked <- read_csv(input_file)

In [ ]:
orig_deg_gene_sets <- list()

for (r in unique(all_genes_ranked$rank)) {
    if (r == 0) {
        next
    }
    
    data <- all_genes_ranked[all_genes_ranked$rank == r,]
    #q <- quantile(data, 0.50, names=FALSE)
    
    orig_deg_gene_sets[[paste0("gene_set_", r)]] <- data$gene_name
}

In [ ]:
length(orig_deg_gene_sets)

### Combine gene sets into "increase lipids" and "decrease lipids"

In [ ]:
deg_gene_sets <- list()

In [ ]:
# genes that increase lipids
deg_gene_sets[["gene_set_increase"]] <- c(
    orig_deg_gene_sets[["gene_set_2"]],
    orig_deg_gene_sets[["gene_set_3"]]
)

In [ ]:
# genes that decrease lipids
deg_gene_sets[["gene_set_decrease"]] <- c(
    orig_deg_gene_sets[["gene_set_-2"]],
    orig_deg_gene_sets[["gene_set_-3"]]
)

In [ ]:
length(deg_gene_sets)

In [ ]:
length(deg_gene_sets[["gene_set_increase"]])

In [ ]:
length(deg_gene_sets[["gene_set_decrease"]])

In [ ]:
# test new increase set
new_set <- deg_gene_sets[["gene_set_increase"]]
expected_set <- union(
    orig_deg_gene_sets[["gene_set_2"]],
    orig_deg_gene_sets[["gene_set_3"]]
)

stopifnot(length(new_set) == length(unique(new_set)))

stopifnot(
    length(new_set) == 
    length(
        intersect(
            new_set,
            expected_set
        )
    )
)

In [ ]:
# test new decrease set
new_set <- deg_gene_sets[["gene_set_decrease"]]
expected_set <- union(
    orig_deg_gene_sets[["gene_set_-2"]],
    orig_deg_gene_sets[["gene_set_-3"]]
)

stopifnot(length(new_set) == length(unique(new_set)))

stopifnot(
    length(new_set) == 
    length(
        intersect(
            new_set,
            expected_set
        )
    )
)

## MultiPLIER Z

In [ ]:
multiplier_z = pd$read_pickle(
    Sys.getenv("PHENOPLIER_MULTIPLIER_MODEL_Z_MATRIX_FILE")
)

In [ ]:
dim(multiplier_z)

In [ ]:
head(multiplier_z)

# Prepare LVs list

In [ ]:
lvs = list()
z_gene_names <- rownames(multiplier_z)

for (cidx in 1:ncol(multiplier_z)) {
    data <- multiplier_z[, cidx]
    names(data) <- z_gene_names
    # q <- quantile(data, 0.75, names=FALSE)
    q <- 0.0
    
    lvs[[paste0("LV", cidx)]] <- data[data > q]
}

# Compute enrichment on all LVs

In [ ]:
n_reps = 10

In [ ]:
set.seed(0)

In [ ]:
results = list()

for (lv in names(lvs)) {
    repetitions = list()
    
    for (i in 1:n_reps) {
        rep_res <- fgsea(pathways = deg_gene_sets, stats = lvs[[lv]], scoreType = "pos", eps = 0.0)[order(pval), ]
        rep_res[, "leadingEdge"] <- sapply(rep_res$leadingEdge, paste, collapse=",")
        rep_res[, "lv"] <- lv
        rep_res[, "rep_idx"] <- i
        
        repetitions[[i]] <- rep_res
    }
    
    res <- do.call(rbind, repetitions)

    results[[lv]] <- res
}

In [ ]:
length(results)

In [ ]:
df <- do.call(rbind, results)

In [ ]:
dim(df)

In [ ]:
head(df)

## Save

In [ ]:
output_file <- file.path(OUTPUT_DIR, "fgsea-all_lvs.tsv")
display(output_file)

In [ ]:
write_tsv(df, output_file)

# Quick analyses/tests

## See how one LV looks like

In [ ]:
df %>% filter(lv == "LV100" & pathway == "gene_set_increase") %>% arrange(desc(padj))

## Show significant LVs

In [ ]:
df_signif <- df %>% group_by(lv, pathway) %>% summarize(max_padj = max(padj)) %>% filter(max_padj < 0.05)

In [ ]:
nrow(df_signif)

In [ ]:
stopifnot(nrow(df_signif) > 500)

In [ ]:
df_signif %>% arrange(max_padj)